# Import data

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Path to dataset files:", path)

import os

DATA_DIR = os.path.join(
    path,
    "asl_alphabet_train",
    "asl_alphabet_train"
)

c:\Users\Windows\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Windows\.cache\kagglehub\datasets\grassknoted\asl-alphabet\versions\1


# Import lib

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


c:\Users\Windows\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


# Load model

# Chuẩn hoá dữ liệu

In [3]:
IMG_SIZE = 64
BATCH_SIZE = 32

In [4]:
datagen = ImageDataGenerator(
    rescale = 1./255, #Chuyển đổi giá trị [0,255] vê [0,1]
    validation_split=0.2 #Chia dữ liệu thanh 80% train và 20% la validation
)

In [5]:
train_data = datagen.flow_from_directory(
    path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset='training' #Chỉ định lấy tập train
)

val_data = datagen.flow_from_directory(
    path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset='validation' #Chỉ định lấy tập validation
)

Found 69623 images belonging to 2 classes.
Found 17405 images belonging to 2 classes.


In [6]:
print(train_data.num_classes)
print(train_data.class_indices)   # hoặc class_names nếu dùng dataset_from_directory


2
{'asl_alphabet_test': 0, 'asl_alphabet_train': 1}


# Khai báo mô hình - fit

In [7]:
model = keras.Sequential(
    [
        keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE,3)),
        keras.layers.MaxPooling2D((2,2)),
        
        keras.layers.Conv2D(64, (3,3), activation='relu'),
        keras.layers.MaxPooling2D((2,2)),
        
        keras.layers.Conv2D(128, (3,3), activation='relu'), 
        keras.layers.MaxPooling2D((2,2)),
        
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation='relu'), #Fully connected layer
        keras.layers.Dropout(0.5), #Khai báo để tránh overfitting
        keras.layers.Dense(train_data.num_classes,activation='softmax')  #29 Classes trong ASL alphabet
    ]
)

model.summary()

c:\Users\Windows\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,454,082 (9.36 MB)

 Trainable params: 2,454,082 (9.36 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# huan luyen
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)


Epoch 1/10
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 177s 80ms/step - accuracy: 0.9995 - loss: 0.0278 - val_accuracy: 0.9997 - val_loss: 0.0107
Epoch 2/10
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 222s 102ms/step - accuracy: 0.9997 - loss: 0.0053 - val_accuracy: 0.9997 - val_loss: 0.0036
Epoch 3/10
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 214s 98ms/step - accuracy: 0.9997 - loss: 0.0119 - val_accuracy: 0.9997 - val_loss: 0.0026
Epoch 4/10
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 185s 85ms/step - accuracy: 0.9997 - loss: 0.0040 - val_accuracy: 0.9997 - val_loss: 0.0026
Epoch 5/10
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 204s 94ms/step - accuracy: 0.9997 - loss: 0.0033 - val_accuracy: 0.9997 - val_loss: 0.0028
Epoch 6/10
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 193s 89ms/step - accuracy: 0.9997 - loss: 0.0038 - val_accuracy: 0.9997 - val_loss: 0.0116
Epoch 7/10
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 186s 85ms/step - accuracy: 0.9996 - loss: 0.0204 - val_accuracy: 0.9997 - val_loss: 0.0046
Epoch 8/10
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 182s 84ms/step - accuracy:

# Save model

In [9]:
model.save("asl_alphabet_model.h5") #.h5:dùn để load lại và dự đoán cho sau  này
print("Saved model")

Saved model


# Đánh giá mô hình

In [10]:
loss, accuracy = model.evaluate(val_data)
print(f"Accuracy: {accuracy:.4f}")
print(f"Loss: {loss:.4f}")

544/544 ━━━━━━━━━━━━━━━━━━━━ 21s 38ms/step - accuracy: 0.9997 - loss: 0.0044
Accuracy: 0.9997
Loss: 0.0044


# Dự đoán

In [11]:
# load mô hình
loaded_model = keras.models.load_model("asl_alphabet_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [12]:
#Chuẩn bị label (chữ cái)
class_names = list(train_data.class_indices.keys())
print(class_names)

['asl_alphabet_test', 'asl_alphabet_train']


In [13]:
# load 1 hinh de xu ly
img_dir = os.path.join(path, "asl_alphabet_train", "asl_alphabet_train", "M")  # duong dan den thu muc chua hinh
# Get first image file
img_file = os.listdir(img_dir)[0]
img_path = os.path.join(img_dir, img_file) # duong dan den hinh can du doan
# img_path = input("Enter the path of the image to predict: ")
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # scale anh
img_array = np.expand_dims(img_array, axis=0)  # them kich thuoc

In [14]:
#Dự đoán
predictions = loaded_model.predict(img_array)
predicted_index = np.argmax(predictions)
predicted_label = class_names[predicted_index] #map index -> label

#print  % các lớp dự đoán
print(predictions)
print(f"Predicted label: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
[[9.5814728e-05 9.9990416e-01]]
Predicted label: asl_alphabet_train
